[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Hawksight-AI/semantica/blob/main/cookbook/advanced/03_Complete_Visualization_Suite.ipynb)

# Complete Visualization Suite

## Overview

Comprehensive visualization capabilities: visualize knowledge graphs, embeddings, analytics, and temporal data.

**Documentation**: [API Reference](https://semantica.readthedocs.io/reference/visualization/)

## Installation

Install Semantica from PyPI:

```bash
pip install semantica
# Or with all optional dependencies:
pip install semantica[all]
```


In [1]:
!pip install -qU semantica


In [2]:
from semantica.visualization import (
    KGVisualizer,
    AnalyticsVisualizer,
    TemporalVisualizer
)
from semantica.kg import GraphBuilder, GraphAnalyzer
import numpy as np


## Step 1: Create Sample Knowledge Graph


In [3]:
builder = GraphBuilder()

entities = [
    {"id": "e1", "type": "Person", "name": "Alice", "properties": {"age": 30}},
    {"id": "e2", "type": "Person", "name": "Bob", "properties": {"age": 35}},
    {"id": "e3", "type": "Organization", "name": "Tech Corp", "properties": {"founded": 2010}},
    {"id": "e4", "type": "Location", "name": "San Francisco", "properties": {"country": "USA"}},
]

relationships = [
    {"source": "e1", "target": "e2", "type": "knows", "properties": {"since": 2020}},
    {"source": "e1", "target": "e3", "type": "works_for", "properties": {"role": "Engineer"}},
    {"source": "e3", "target": "e4", "type": "located_in", "properties": {}},
]

knowledge_graph = builder.build(entities, relationships)


Status,Action,Module,Submodule,File,Time
🔄,Semantica is building,🧠 kg,GraphBuilder,-,0.00s


## Step 2: Knowledge Graph Visualization


In [4]:
kg_visualizer = KGVisualizer(layout="force", color_scheme="vibrant")
kg_visualizer.visualize_network(knowledge_graph, output="interactive")

## Step 3: Graph Analytics Visualization


In [5]:
graph_analyzer = GraphAnalyzer()

centrality_results = graph_analyzer.calculate_centrality(
    knowledge_graph, 
    centrality_type="degree"
)

centrality_scores = {}
if centrality_results and "centrality_measures" in centrality_results:
    degree_centrality = centrality_results["centrality_measures"].get("degree", {})
    if isinstance(degree_centrality, dict) and "centrality" in degree_centrality:
        centrality_scores = degree_centrality["centrality"]
    elif isinstance(degree_centrality, dict):
        centrality_scores = degree_centrality

communities_result = graph_analyzer.detect_communities(
    knowledge_graph, 
    algorithm="louvain"
)

communities = []
community_dict = {}
if communities_result and "communities" in communities_result:
    communities_data = communities_result["communities"]
    if isinstance(communities_data, list):
        communities = communities_data
        for idx, community in enumerate(communities):
            if isinstance(community, list):
                for node in community:
                    community_dict[node] = idx
            elif isinstance(community, dict) and "nodes" in community:
                for node in community["nodes"]:
                    community_dict[node] = idx

analytics_visualizer = AnalyticsVisualizer()
analytics_visualizer.visualize_centrality(centrality_scores, title="Node Centrality Scores")

if community_dict:
    analytics_visualizer.visualize_communities(
        knowledge_graph, 
        community_dict, 
        title="Community Detection"
    )


## Step 5: Temporal Data Visualization


In [6]:
import datetime
from semantica.visualization import TemporalVisualizer
import pandas as pd
import numpy as np

# 1. Setup Data: AI Research Lab Evolution (2020-2024)
# This dataset simulates a growing network of researchers, papers, and grants

start_date = datetime.date(2020, 1, 1)

# Entities with lifespans
entities = [
    {"id": "Lab_Alpha", "type": "Organization", "start": "2020-01-01", "end": "2024-12-31", "properties": {"budget": "High"}},
    {"id": "Dr_Smith", "type": "Researcher", "start": "2020-01-15", "end": "2024-12-31", "properties": {"h_index": 15}},
    {"id": "Dr_Jones", "type": "Researcher", "start": "2020-03-01", "end": "2024-12-31", "properties": {"h_index": 12}},
    {"id": "Paper_X", "type": "Publication", "start": "2020-11-20", "end": "2024-12-31", "properties": {"citations": 50}},
    {"id": "Grant_A", "type": "Funding", "start": "2021-01-01", "end": "2022-12-31", "properties": {"amount": 1000000}},
    {"id": "Dr_Chen", "type": "Researcher", "start": "2021-06-01", "end": "2024-12-31", "properties": {"h_index": 8}},
    {"id": "Paper_Y", "type": "Publication", "start": "2022-03-15", "end": "2024-12-31", "properties": {"citations": 25}},
    {"id": "Startup_Beta", "type": "SpinOff", "start": "2023-01-01", "end": "2024-12-31", "properties": {"valuation": "5M"}},
]

# Relationships with timestamps
relationships = [
    {"source": "Dr_Smith", "target": "Lab_Alpha", "type": "WORKS_AT", "timestamp": "2020-01-15"},
    {"source": "Dr_Jones", "target": "Lab_Alpha", "type": "WORKS_AT", "timestamp": "2020-03-01"},
    {"source": "Dr_Smith", "target": "Paper_X", "type": "AUTHORED", "timestamp": "2020-11-20"},
    {"source": "Dr_Jones", "target": "Paper_X", "type": "AUTHORED", "timestamp": "2020-11-20"},
    {"source": "Lab_Alpha", "target": "Grant_A", "type": "RECEIVED", "timestamp": "2021-01-01"},
    {"source": "Dr_Chen", "target": "Lab_Alpha", "type": "WORKS_AT", "timestamp": "2021-06-01"},
    {"source": "Dr_Chen", "target": "Paper_Y", "type": "AUTHORED", "timestamp": "2022-03-15"},
    {"source": "Dr_Smith", "target": "Paper_Y", "type": "AUTHORED", "timestamp": "2022-03-15"},
    {"source": "Lab_Alpha", "target": "Startup_Beta", "type": "SPUN_OFF", "timestamp": "2023-01-01"},
    {"source": "Dr_Jones", "target": "Startup_Beta", "type": "CTO", "timestamp": "2023-02-01"},
]

# Metrics over time
dates = pd.date_range(start="2020-01-01", end="2024-01-01", freq="M")
metrics = {
    "dates": [d.strftime("%Y-%m-%d") for d in dates],
    "funding_usd": [100000 + (i * 50000) + (np.random.randint(-10000, 10000)) for i in range(len(dates))],
    "team_size": [2 + int(i/5) for i in range(len(dates))],
    "publications": [int(i/4) for i in range(len(dates))]
}

# 4. Generate Timestamps Map (Required for TemporalVisualizer)
# This maps each entity to the specific time points where it is "active" or relevant
timestamps = {}

# Collect all relevant dates (monthly granularity)
all_dates = [d.strftime("%Y-%m-%d") for d in dates]

for entity in entities:
    eid = entity["id"]
    start = entity.get("start")
    end = entity.get("end")
    
    # In a real app, you'd calculate overlap. Here we'll just assign all dates 
    # that fall within the entity's lifespan
    entity_times = [d for d in all_dates if start <= d <= end]
    timestamps[eid] = entity_times
    
temporal_kg = {
    "entities": entities,
    "relationships": relationships,
    "metrics": metrics,
    "timestamps": timestamps
}

# 2. Initialize Visualizer
viz = TemporalVisualizer()

print("1. Generating Temporal Dashboard...")
# This creates a combined view of lifecycles, activity, and metrics
dashboard = viz.visualize_temporal_dashboard(
    temporal_kg,
    title="AI Research Lab Evolution (2020-2024)",
    output="interactive"
)
dashboard.show()

print("2. Generating Network Evolution Animation...")
# This creates a playable animation of the network graph
animation = viz.visualize_network_evolution(
    temporal_kg,
    title="Network Growth Over Time",
    output="interactive"
)
animation.show()

print("3. Generating Timeline View...")
timeline = viz.visualize_timeline(
    temporal_kg,
    title="Entity Lifecycles",
    output="interactive"
)
timeline.show()

1. Generating Temporal Dashboard...


2. Generating Network Evolution Animation...


3. Generating Timeline View...


## Summary

All visualization types demonstrated:
- Knowledge Graph Visualization
- Embedding Visualization (t-SNE)
- Graph Analytics Visualization (Centrality & Communities)
- Temporal Data Visualization (Timeline & Evolution)
